<a href="https://colab.research.google.com/github/PANDASANG1231/deeplearn_note/blob/master/024_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Note
  -  The result of Lenet start a new idea of Conv
  -  The result of AlexNet prove that we need to deploy deeper and wider Network
  -  The result of VGG shows that we can make the CNN more standard and easy to replicate




In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/deeplearning_note')
from tool import *

Mounted at /content/drive


In [ ]:
import torch
import torchvision

In [ ]:
! nvidia-smi

Tue Nov  9 04:17:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class VGG_block(torch.nn.Module):

  def __init__(self, num_conv, in_channels, out_channels):

    super().__init__()
    layer = []
    for i in range(num_conv):
        layer.append(torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding='same'))
        layer.append(torch.nn.ReLU())
        in_channels = out_channels
    layer.append(torch.nn.MaxPool2d(kernel_size=2, stride=2))

    self.model = torch.nn.Sequential(*layer)

  def forward(self, X):

    return self.model(X)


classic_vgg_arch = [(1, 64), (1, 128), (2, 256), (2, 512), (2, 512)]
class VGG(torch.nn.Module):

  def __init__(self, in_channel_0, vgg_arch):

    super().__init__()

    model = []
    in_channels = in_channel_0
    for num_conv, out_channels in classic_vgg_arch:
        model.append(VGG_block(num_conv=num_conv, out_channels=out_channels, in_channels=in_channels))
        in_channels = out_channels

    self.model = torch.nn.Sequential(*model, torch.nn.Flatten(),
        torch.nn.Linear(in_features=out_channels*7*7, out_features=4096), torch.nn.ReLU(), torch.nn.Dropout(p=0.5),
        torch.nn.Linear(in_features=4096, out_features=4096), torch.nn.ReLU(), torch.nn.Dropout(p=0.5),
        torch.nn.Linear(in_features=4096, out_features=10),) 

  def forward(self, X):

    return self.model(X)



x = torch.randn(size=(1, 1, 224, 224))
vgg = VGG(vgg_arch=classic_vgg_arch, in_channel_0=1)
for layer in vgg.model:
  x = layer(x)
  print(layer.__class__.__name__, x.shape) 


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


VGG_block torch.Size([1, 64, 112, 112])
VGG_block torch.Size([1, 128, 56, 56])
VGG_block torch.Size([1, 256, 28, 28])
VGG_block torch.Size([1, 512, 14, 14])
VGG_block torch.Size([1, 512, 7, 7])
Flatten torch.Size([1, 25088])
Linear torch.Size([1, 4096])
ReLU torch.Size([1, 4096])
Dropout torch.Size([1, 4096])
Linear torch.Size([1, 4096])
ReLU torch.Size([1, 4096])
Dropout torch.Size([1, 4096])
Linear torch.Size([1, 10])


In [ ]:
transforms = torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Resize(size=224)
             ])

train_dataset = torchvision.datasets.FashionMNIST(root='./', train=True, download=True, transform=transforms)
test_dataset = torchvision.datasets.FashionMNIST(root='./', train=False, download=True, transform=transforms)

In [ ]:
batch_size = 128
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=batch_size,
                                              shuffle=False)

device = torch.device('cuda')
model = VGG(vgg_arch=[(x, x1//4) for x,x1 in classic_vgg_arch], in_channel_0=1)
loss = torch.nn.CrossEntropyLoss()

print("This is a simple VGG-11...")
x = torch.randn(size=(1, 1, 224, 224))
for layer in model.model:
  x = layer(x)
  print(layer.__class__.__name__, x.shape) 



This is a simple VGG-11...
VGG_block torch.Size([1, 64, 112, 112])
VGG_block torch.Size([1, 128, 56, 56])
VGG_block torch.Size([1, 256, 28, 28])
VGG_block torch.Size([1, 512, 14, 14])
VGG_block torch.Size([1, 512, 7, 7])
Flatten torch.Size([1, 25088])
Linear torch.Size([1, 4096])
ReLU torch.Size([1, 4096])
Dropout torch.Size([1, 4096])
Linear torch.Size([1, 4096])
ReLU torch.Size([1, 4096])
Dropout torch.Size([1, 4096])
Linear torch.Size([1, 10])


In [ ]:
train_p2(epoch_num=10, 
         model=model,  
         loss=loss, 
         lr=0.05, 
         train_data_iter=train_dataloader,
         test_data_iter=test_dataloader,
         device=device,
         optim_type="SGD")

loss 0.983, train acc 0.635, test acc 0.835
loss 0.352, train acc 0.869, test acc 0.884
loss 0.277, train acc 0.897, test acc 0.893
loss 0.238, train acc 0.911, test acc 0.900
loss 0.203, train acc 0.925, test acc 0.902
loss 0.175, train acc 0.935, test acc 0.907
loss 0.146, train acc 0.945, test acc 0.915
loss 0.121, train acc 0.956, test acc 0.917
loss 0.094, train acc 0.966, test acc 0.919
loss 0.073, train acc 0.973, test acc 0.922
Calculation Ability: 12327.9 examples/sec on cuda
